In [1]:
import sys
import os
from pathlib import Path

from IPython.display import display, Markdown
from openai import OpenAI
from dotenv import dotenv_values

# Add the parent directory to sys.path so Python can find the Utils module
sys.path.append(os.path.abspath(".."))

from Utils import (
    get_guidance_and_feedback,
    get_questions_and_answers,
    create_content,
)

# Get required environment variables
config = dotenv_values("../.env")
open_api_key = config.get("OPENAI_API_KEY", None)
tutor_agent_id = config.get("TUTOR_AGENT_ID", None)

# Check if the environment variables are set
if not (tutor_agent_id and open_api_key):
    print("Need TUTOR_AGENT_ID and OPEN_API_KEY in the .env file")
    sys.exit(1)

# Create the OpenAI client
client = OpenAI(api_key=config["OPENAI_API_KEY"])

Generate the content for our prompt message to our bot.  Our prompt must look like the input examples we provided to our assistant in its instructions.  For this toy example we are using JSON files of question data and student answers as a store.

In [2]:
q_path = Path('../Data/question_store.json')
a_path = Path('../Data/student_answers.json')
questions, answers = get_questions_and_answers(q_path, a_path)

Having loaded the question and answer data we can now create input content.

In [3]:
question_id = "Q1"
student_id = "student_4"

question = questions[question_id]
student_answer = answers[question_id][student_id]

content = create_content(question, student_answer)
print(content)

Input:
Exam Question: "Can you observe the fundamental transition for Hydrogen gas using infrared spectroscopy? Explain your answer."
Student Answer: "To observe an H gas transition with IR a high-powered IR needs to be used since its transition energy is high."
Concepts Tested: 
	- Molecular Symmetry and Dipole Moment
	- Vibrational Transitions in IR Spectroscopy
	- Coupling of electromagnetic field oscillations with vibrations in the molecule
Example Good Answer: "Infrared spectroscopy relies on the interaction between molecular vibrations and the oscillating electric field of infrared light, requiring a molecule to have a dipole moment. During a vibrational transition, if this dipole moment changes, it can couple with the IR light's electric field, and this coupling allows the light to affect the dipole moment of the molecule. The effectiveness of this coupling, and thus the intensity of IR absorption, depends on the matching of the IR light's frequency with the molecule's vibration

Now we start a new conversation thread and send our content as a messages to our assistant.  This can take some time to run.

In [4]:
thread, messages, run = get_guidance_and_feedback(
    client, content, tutor_agent_id, temperature=0.1, thread_id=None, show_run=True)

**Bot Message**:


Your answer suggests that using a high-powered IR source might enable the observation of a fundamental transition in hydrogen gas. However, this is not accurate in the context of infrared spectroscopy and the properties of hydrogen gas.

Infrared spectroscopy relies on the interaction between molecular vibrations and the oscillating electric field of infrared light. For a molecule to absorb IR radiation, it must have a dipole moment that changes during vibration. Hydrogen gas ($H_2$), being a homonuclear diatomic molecule, has no permanent dipole moment due to its symmetric electronic distribution. This symmetry means that during vibrational transitions, there is no change in dipole moment, and thus no coupling with the IR light's electric field. Therefore, regardless of the power of the IR source, $H_2$ cannot absorb IR radiation because there is no dipole moment change to interact with the electromagnetic field.

To improve your understanding, consider reviewing the following sections in the textbook "Quantum Chemistry" by Donald A. Macquarrie:
- **Chapter 5.6**: The Harmonic Oscillator Accounts for the Infrared Spectrum of a Diatomic Molecule. This section will help you understand how molecular vibrations relate to IR spectroscopy.
- **Chapter 3.4**: Molecular Symmetry and Dipole Moment. This will clarify why homonuclear diatomic molecules like $H_2$ do not have a dipole moment and thus do not show IR activity.

Understanding these concepts will help you better explain why certain molecules are inactive in IR spectroscopy and how molecular properties influence spectroscopic techniques.



**Run object attributes**:
- status: completed
- temperature: 0.1
- tools: [FileSearchTool(type='file_search')]
- usage: Usage(completion_tokens=322, prompt_tokens=3389, total_tokens=3711)


Now we have started the thread, we can continue if we want the chat with the bot.

In [5]:
follow_up = "Which section of the textbook explains the second postulate of quantum mechanics?"
thread, messages, run = get_guidance_and_feedback(
    client, follow_up, tutor_agent_id, temperature=0.1, thread_id=thread, show_run=True)

**Bot Message**:


The second postulate of quantum mechanics, which deals with the quantization of physical properties and the role of operators in quantum mechanics, is discussed in detail in Chapter 4 of the textbook "Quantum Chemistry" by Donald A. Macquarrie.

Specifically, you should review:
- **Chapter 4.2**: Quantum-Mechanical Operators and Observables. This section explains how physical observables are represented by operators in quantum mechanics, which is a core aspect of the second postulate.

This section will provide a comprehensive understanding of how operators are used to describe measurable quantities in quantum mechanics and how these operators relate to the eigenvalues and eigenstates of the system.



**Run object attributes**:
- status: completed
- temperature: 0.1
- tools: [FileSearchTool(type='file_search')]
- usage: Usage(completion_tokens=138, prompt_tokens=3731, total_tokens=3869)
